# 读取ICESAT-2数据进行分析
此笔记本（{nb-download}`下载<is2_data_readin.ipynb>`）说明了使用Icepyx用于读取ICESAT-2数据文件的使用，并将其加载到数据对象中。
当前，默认数据对象是Xarray数据集，持续的工作以提供其他数据对象类型的支持。

有关如何订购和下载ICESAT-2数据的更多信息，请参见[Icepyx Data Access教程]（https://icepyx.readthedocs.io/en/latest/latest/example_notebooks/is2_data_access.html）。

＃＃＃ 动机
最常见的是，当您打开数据文件时，必须指定基础数据结构以及如何读取信息。
一个简单的示例，例如，在打开CSV或类似划界的文件时，请让软件知道数据是否包含标头行，数据类型是什么（字符串，double，float，boolean等），分区是什么，以及要加载哪些列或行。
许多ICESAT-2数据读取器本质上是相当手动的，要求您准确地输入各种数据变量的字符串路径列表。

Icepyx通过依靠其对ICESAT-2特定数据文件变量存储结构的认识来简化此过程。
您可以为“读取”对象提供一些选项，而Icepyx将为您提供一些选择（如本笔记本中所详细介绍），您无需手动迭代光束对。

＃＃＃ 方法
如果您对引擎盖下发生的事情感兴趣：Icepyx使用[Xarray]（https://docs.xarray.dev/en/stable/）库中读取数据集的每个请求变量。Icepyx格式化每个请求的变量，然后将每个变量的读取数据合并以创建一个单个数据对象。Xarray的使用功能很强大，因为返回的数据对象可以与相关的Xarray处理工具一起使用。

Import packages, including icepyx

In [ ]:
import icepyx as ipx

---------------------------------

## Quick-Start Guide
对于那些可能正在尝试玩这个的人（但不希望所有细节/解释）

In [ ]:
path_root = '/full/path/to/your/ATL06_data/'
reader = ipx.Read(path_root)

In [ ]:


reader.vars.append(beam_list=['gt1l', 'gt3r'], var_list=['h_li', "latitude", "longitude"])

In [ ]:
ds = reader.load()
ds

In [ ]:
ds.plot.scatter(x="longitude", y="latitude", hue="h_li", vmin=-100, vmax=2000)

-------------------------------------------------------
## 加载的关键步骤（读取）ICESAT-2数据

在使用Icepyx的ICESAT-2数据中阅读以几个简单的步骤进行：
1.让Icepyx知道在哪里可以找到您的数据（这可能是本地文件或云存储中数据的URL）
2.创建一个icepyx`read`对象
3.列出您要阅读的变量的列表（不适用于网格产品）
4.将数据加载到内存中（如果您使用的DASK，请懒惰地阅读）

我们在此笔记本中更详细地详细介绍了这些步骤。

### Step 0: Get some data if you haven't already
如果您在本地系统上还没有一些数据文件，则有几行代码可以为您设置一些数据文件。

In [ ]:
region_a = ipx.Query('ATL06',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59')

In [ ]:
region_a.download_granules(path=path_root)

```{invernition}重要身份验证更新
以前，Icepyx要求您明确使用`.earthdata_login（）`函数登录。运行此功能被弃用并会导致错误，因为Icepyx会根据需要调用登录功能。用户仍然需要提供其凭据。
```

### 步骤1：设置数据源路径

提供了要阅读的数据的完整途径（即打开）。
目前接受的输入是：
*目录的字符串路径 - 目录中的所有文件将打开
*单个文件的字符串路径 - 将打开一个文件
* filepaths的列表 - 列表中的所有文件将打开
*一个球字符串（请参阅[GLOL]（https://docs.python.org/3/library/glob.html）） - 任何匹配Glog模式的文件都将打开

In [ ]:
path_root = '/full/path/to/your/data/'

In [ ]:
# filepath = path_root + 'ATL06-20181214041627-Sample.h5'

In [ ]:
# list_of_files = ['/my/data/ATL06/processed_ATL06_20190226005526_09100205_006_02.h5', 
#                  '/my/other/data/ATL06/processed_ATL06_20191202102922_10160505_006_01.h5']

#### Glob Strings

[glob](https://docs.python.org/3/library/glob.html)是一个python库，允许用户在其文件系统中列出匹配给定模式的文件系统中的文件。Icepyx使用Glob库为用户提供更大的灵活性对输入文件列表。

Glob使用``*'''？'作为通配符的字符来工作，其中*````*''匹配任何数量的字符，并且'？？'匹配一个字符。例如：

*`/this/path/*。h5`：指`/this/path“文件夹中的所有`.h5`文件（示例匹配：“/this/path/path/processed_atl03_2019113113021008_099999930303030303030330303_006_006_01.h5）myfavoriteicsat-2file.h5”）
*`/this/path/*atl07*.h5`：指`/this/path“文件夹中具有atl07的所有`.h5`文件。（示例匹配：“/this/path/atl07-02_20221012220720720_03391701_005_01.h5”或“/this/path/path/processed_atl07.h5”）
*`/this/path/atl ??/*。h5`：指在`/this/path'的子文件夹中的所有`.h5`文件and'atl的子目录中，然后是任何2个字符（示例）匹配：“/this/path/atl03/processed_atl03_2019113021008_09930303030303_006_01.h5”，“” /this/path/atl06/myfile.h5）

有关更多深度说明或高级环球路径，例如角色类和范围，请参见Glob文档或其他在线解释器教程。

#### 递归目录搜索

默认情况下，Glob不会搜索所有子目录以匹配的filepaths，但它具有这样做的能力。

如果您想递归搜索，则可以通过以下任务来实现这一目标：
1.将`递归的论点都传递到`glob_kwargs'中，包括`\ ** \`在您的filepath中
2.直接使用Glob来创建filepath的列表

这两种方法中的每一种如下所示。

Method 1: passing the `recursive` argument into `glob_kwargs`

In [ ]:
ipx.Read('/path/to/**/folder', glob_kwargs={'recursive': True})

You can use `glob_kwargs` for any additional argument to Python's builtin `glob.glob` that you would like to pass in via icepyx.

方法2：直接使用Glob来创建filepath的列表

In [ ]:
import glob

In [ ]:
list_of_files = glob.glob('/path/to/**/folder', recursive=True)
ipx.Read(list_of_files)

````{invernition}读取模块更新
以前，Icepyx需要两个其他条件：1）a“产品”参数和2）您的文件要么匹配默认的filename_pattern`，要么用户提供了自己的`filename_pattern`。这两个要求已被删除。现在，“ product”直接从文件元数据（root组的`short_name`属性）中读取。通过“ FILENAME_PATTERN”指定多个文件的灵活性已被[Glob String]（https://docs.python.org/3/library/glob.html）替换。

现在已弃用了“ product”和“ filename_pattern”参数，并在Icepyx版本1.0.0中删除。
````

### Step 2: Create an icepyx read object

Using the `data_source` described in Step 1, we can create our Read object.

In [ ]:
reader = ipx.Read(data_source=path_root)

现在，读取对象包含匹配文件的列表，这些文件最终将加载到Python中。您可以直接在读取对象上检查其属性，例如所定位的文件或已识别产品。

In [ ]:
reader.filelist

In [ ]:
reader.product

### Step 3: Specify variables to be read in

要将数据加载到内存或准备分析中，Icepyx需要知道您想阅读哪些变量。
如果您使用Icepyx从NSIDC下载具有可变子集的数据（这是默认值），那么您可能已经熟悉Icepyx`变量模块以及如何创建和修改变量的列表。
我们在这里展示一个特定情况，但我们鼓励您查看[Icepyx变量示例](https://icepyx.readthedocs.io/en/latest/example/example_notebooks/is2_data_variables.html)操纵ICESAT-2变量路径的列表（为多个数据产品提供了示例）。

如果您想查看所有可用的所有路径 +变量组合的\ [可能很长的\]列表，则该列表中的第一个文件默认是从列表中的第一个文件生成的，此不合格（不变）列表是生成的（因此，并非所有变量都可以包含在所有文件，具体取决于您的访问方式）。

In [ ]:
reader.vars.avail()

为了使事情变得更容易，您可以使用Icepyx的内置默认列表，该默认列表为您的非网格数据产品加载常用的变量，也可以创建自己要读取的变量列表。
Icepyx将通过从一个源文件之一创建列表来确定哪些变量可供阅读。
如果您有多个正在阅读的文件，Icepyx将自动生成文件名列表，并将第一个文件列出以获取可用变量的列表。

因此，如果您在文件之间有不同的变量（甚至来自同一数据产品），则可能会遇到问题并需要提出解决方法（我们可以帮助您这样做！）。
我们预计大多数用户将拥有他们寻求在所有数据文件中加载的最小变量集，因此我们目前尚未开发此功能。
如果这对您有用，或者您遇到此问题，请与我们取得联系！

您可以为Gridded ICESAT-2产品创建一个变量列表。但是，文件中的所有变量仍将添加到您的数据集中。（这是我们目前正在探索扩展方面的领域 - 如果您正在努力并想贡献！）

对于基本情况，假设我们希望所有光束对的高度，纬度和经度阅读。
我们将变量列表创建为

In [ ]:
reader.vars.append(var_list=['h_li', "latitude", "longitude"])

Then we can view a dictionary of the variables we'd like to read in.

In [ ]:
reader.vars.wanted

不要忘记 - 如果您需要重新开始，并重新生成所需的变量列表，这很容易！

In [ ]:
reader.vars.remove(all=True)

### Step 4: Loading your data

现在，您已经设置了所有选项，就可以将您的ICESAT-2数据读取到内存中！

In [ ]:
ds = reader.load()

在Jupyter笔记本中，您可以获得数据对象的摘要视图。

***注意：Icepyx通过为每个输入颗粒创建Xarray数据集，然后合并它们来加载数据。在某些情况下，自动合并失败，需要手动处理。在这些情况下，Icepyx将返回带有失败Xarray合并的错误消息的警告和颗粒数据集列表***

如果您无意间多次提供不同的文件名或RGT+循环自动生成的`gran_idx值匹配的rgt+循环中的分段产品中，这可能会发生这种情况。在后一种情况下，您可以简单地为`ds`'中的每个数据集提供唯一的“ gran_idx”值，然后运行`import x array作为xr`和ds_merged = xr.merge（ds）`创建一个合并的数据集。

In [ ]:
ds

## 进行数据分析！

从这里，您可以开始分析。
最终，Icepyx的目的是包括带有ICESAT-2 Aware功能的Xarray扩展程序，使您可以轻松地仅使用来自强光束的数据，例如。
该功能仍在开发中。
为了娱乐，我们提供了一个使用Xarray的内置功能制成的基本情节。

In [ ]:
ds.plot.scatter(x="longitude", y="latitude", hue="h_li", vmin=-100, vmax=2000)

向用户的开发人员注释：
我们的下一步将是创建具有ICESAT-2 AREATION功能的Xarray扩展程序（例如“ get_strong_beams”等）。
如果您有任何想法或已经开发了功能（我们可以与您合作添加它们，或为您添加它们！）。

#### Credits
* original notebook by: Jessica Scheick
* notebook contributors: Wei Ji and Tian